In [1]:
import sys
sys.path.append("..")

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import xgboost
from sklearn.ensemble import RandomForestClassifier
from satreeverify import SATreeAttack

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = load_breast_cancer()
X = data['data']
y = data["target"]
feature_names = data["feature_names"]
y = y==1

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# XGBoost Model

In [5]:
ntrees = 50
clf = xgboost.XGBClassifier(n_estimators=ntrees, max_depth=3).fit(x_train, y_train)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:20:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9473684210526315


In [6]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.9807157 , 0.01928429]], dtype=float32)

In [7]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,13.110000,0.000000,"[-inf, 14.5149994]"
1,15.560000,15.519465,0.002605,"[-inf, 15.5349998]"
2,87.210000,87.210000,0.000000,"[-inf, 97.6800003]"
3,530.200000,530.200000,0.000000,"[-inf, 696.25]"
4,0.139800,0.086493,0.381306,"[-inf, 0.0865800008]"
5,0.176500,0.176500,0.000000,"[0.0714350045, inf]"
6,0.207100,0.207100,0.000000,"[0.119199999, inf]"
7,0.096010,0.096010,0.000000,"[0.0492300019, inf]"
8,0.192500,0.192500,0.000000,"[0.166449994, inf]"
9,0.076920,0.076920,0.000000,"[-inf, inf]"


In [8]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

In [9]:
result = attack.hard_attack(sample, epsilon, 8)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,14.529514,0.108277,"[14.5149994, inf]"
1,15.560000,15.519465,0.002605,"[-inf, 15.5349998]"
2,87.210000,97.777680,0.121175,"[97.6800003, inf]"
3,530.200000,530.200000,0.000000,"[-inf, 696.25]"
4,0.139800,0.139800,0.000000,"[0.0901449993, inf]"
5,0.176500,0.176500,0.000000,"[0.0714350045, inf]"
6,0.207100,0.207100,0.000000,"[0.119199999, inf]"
7,0.096010,0.096010,0.000000,"[0.0492300019, inf]"
8,0.192500,0.166284,0.136189,"[-inf, 0.166449994]"
9,0.076920,0.076920,0.000000,"[-inf, inf]"


In [10]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

# RandomForest Model

In [11]:
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

0.9824561403508771


In [12]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.78643988, 0.21356012]])

In [13]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,14.700255,0.121301,"[14.699999809265137, 14.954999923706055]"
1,15.560000,14.105880,0.093452,"[-inf, 14.119999885559082]"
2,87.210000,71.813867,0.176541,"[70.68000030517578, 71.81500244140625]"
3,530.200000,558.767644,0.053881,"[558.6499938964844, 676.2999877929688]"
4,0.139800,0.117471,0.159719,"[0.08875500038266182, 0.11750000342726707]"
5,0.176500,0.124533,0.294428,"[0.107999999076128, 0.124549999833107]"
6,0.207100,0.207100,0.000000,"[0.13795000314712524, inf]"
7,0.096010,0.064130,0.332052,"[0.06381000205874443, 0.06413000077009201]"
8,0.192500,0.160690,0.165244,"[0.15114999562501907, 0.1607000008225441]"
9,0.076920,0.076920,0.000000,"[0.060009999200701714, inf]"


In [14]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

In [15]:
result = attack.hard_attack(sample, epsilon, 8)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,13.110000,0.000000,"[10.405499935150146, 13.139999866485596]"
1,15.560000,14.105880,0.093452,"[-inf, 14.119999885559082]"
2,87.210000,88.380059,0.013417,"[88.36999893188477, 98.43000030517578]"
3,530.200000,530.200000,0.000000,"[-inf, 558.6499938964844]"
4,0.139800,0.117464,0.159768,"[0.08194000273942947, 0.11750000342726707]"
5,0.176500,0.176500,0.000000,"[0.124549999833107, inf]"
6,0.207100,0.137931,0.333986,"[0.11934999749064445, 0.13795000314712524]"
7,0.096010,0.096010,0.000000,"[0.07409500330686569, inf]"
8,0.192500,0.173487,0.098768,"[0.1607000008225441, 0.17350000143051147]"
9,0.076920,0.076920,0.000000,"[0.05671999976038933, inf]"


In [16]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))